In [ ]:
# Import libraries
import pandas as pd
import pickle
import numpy as np
import itertools
import math
import matplotlib.pyplot as plt
from datetime import datetime
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
import tensorflow as tf
import random
from keras.callbacks import TensorBoard, EarlyStopping, ModelCheckpoint
from sklearn.preprocessing import StandardScaler, RobustScaler
from tensorflow.keras.optimizers import Adam
from keras_tuner import RandomSearch
from keras_tuner.engine.hyperparameters import HyperParameters
from sklearn.metrics import mean_squared_error, mean_absolute_error, accuracy_score, make_scorer, confusion_matrix, ConfusionMatrixDisplay, roc_auc_score
from sklearn.model_selection import cross_val_score, RepeatedKFold, RandomizedSearchCV, KFold, StratifiedKFold
from xgboost import XGBRegressor, XGBClassifier
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier
import statsmodels.api as sm
from statsmodels.stats.outliers_influence import variance_inflation_factor
from statsmodels.stats.multitest import multipletests
from kneed import KneeLocator
from sklearn.cluster import KMeans
from imblearn.combine import SMOTETomek
from imblearn.under_sampling import TomekLinks
from imblearn.over_sampling import SMOTE
import seaborn as sns
from statsmodels.stats.diagnostic import het_breuschpagan
from statsmodels.stats.stattools import durbin_watson
from scipy.stats import norm
from sklearn.svm import SVR
from scipy.stats import shapiro
import shap
from xgboost import XGBRegressor

# Linear regression

### Data recurring

In [ ]:
# Import data
with open("Data final//data_recurring_imputed.pkl", "rb") as file:
    data = pickle.load(file)

In [ ]:
# Split data into dependent and independent variables
X = data.drop(["Total_revenue"], axis = "columns")
y = data["Total_revenue"]
# Split data into training and test sets
train_X, test_X, train_y, test_y = train_test_split(X, y, test_size = 0.25, random_state = 3)

In [ ]:
# Drop insignificant and multicollinear variables 
# This is the result of iteratively running the model and removing insignificant and multicollinear variables
train_X = train_X.drop(["Quarter_2", "Number_account", "Exchange rate", "Consumption of durables", "Unemployment rate", "Interest rate", "GDP growth", "Quarter_4", "Limit_AMT_max", "Age", "Consumer confidence index", "Limit_AMT_min", "Inflation"], axis = "columns")

In [ ]:
# Add a constant
train_X = sm.add_constant(train_X)
# Train a Linear Regression model
model_linear = sm.OLS(train_y, train_X).fit(cov_type='HC3')

In [ ]:
# Calculate VIF
vif = pd.DataFrame([variance_inflation_factor(train_X.drop(["const"], axis = "columns").values, i) for i in range(train_X.drop(["const"], axis = "columns").shape[1])], index=train_X.drop(["const"], axis = "columns").columns, columns=['VIF'])
print('Variance Inflation Factors:')
print(vif.round(2))

In [ ]:
# Check significant variables and coefficients
print(model_linear.summary())

In [ ]:
# Extract p-values
p_values = model_linear.pvalues[1:] # exclude the intercept

# Apply Bonferroni correction
alpha = 0.05
reject, adjusted_p_values, _, _ = multipletests(p_values, alpha=alpha, method='bonferroni')

# Print the results
print('P-values:', p_values)
print('Adjusted p-values:', adjusted_p_values)
print('Rejected hypotheses:', reject)

In [ ]:
# Run Breusch-Pagan test for heteroskedasticity
bp_test = het_breuschpagan(model_linear.resid, model_linear.model.exog)
print('Breusch-Pagan test p-value:', bp_test[1]) # heteroskedastic

# Run Durbin-Watson test for autocorrelation
dw_test = durbin_watson(model_linear.resid)
print('Durbin-Watson test statistic:', dw_test)
print('Durbin-Watson test p-value:', 2 * norm.cdf(-abs(dw_test - 2))) # no autocorrelation

In [ ]:
# Prepare test data set
test_X = test_X.drop(["Quarter_2", "Number_account", "Exchange rate", "Consumption of durables", "Unemployment rate", "Interest rate", "GDP growth", "Quarter_4", "Limit_AMT_max", "Age", "Consumer confidence index", "Limit_AMT_min", "Inflation"], axis = "columns")
test_X = sm.add_constant(test_X)
# Calculate perdictions for test data set
pred = model_linear.predict(test_X)

In [ ]:
# Calculate MAE
print(f"MAE: {mean_absolute_error(test_y, pred):.4f}")

In [ ]:
# Calculate MSE 
print(f"MSE: {mean_squared_error(test_y, pred):.4f}")

In [ ]:
# Calculate RMSE
print(f"RMSE: {mean_squared_error(test_y, pred, squared=False):.4f}")

### Data new

In [ ]:
# Import data
with open("Data final//data_new_imputed.pkl", "rb") as file:
    data = pickle.load(file)

In [ ]:
# Split data into dependent and independent variables
X = data.drop(["Total_revenue"], axis = "columns")
y = data["Total_revenue"]
# Split data into training and test sets
train_X, test_X, train_y, test_y = train_test_split(X, y, test_size = 0.25, random_state = 3)

In [ ]:
# Drop insignificant and multicollinear variables 
# This is the result of iteratively running the model and removing insignificant and multicollinear variables
train_X = train_X.drop(["GDP growth", "Consumption of durables", "Inflation", "Quarter_2", "Exchange rate", "Unemployment rate", "Quarter_3", "Consumer confidence index", "Interest rate", "Age"], axis = "columns")

In [ ]:
# Add a constant
train_X = sm.add_constant(train_X)
# Train a Linear Regression model
model_linear = sm.OLS(train_y, train_X).fit(cov_type='HC3')

In [ ]:
# Calculate VIF
vif = pd.DataFrame([variance_inflation_factor(train_X.drop(["const"], axis = "columns").values, i) for i in range(train_X.drop(["const"], axis = "columns").shape[1])], index=train_X.drop(["const"], axis = "columns").columns, columns=['VIF'])
print('Variance Inflation Factors:')
print(vif.round(4))

In [ ]:
# Check significant variables and coefficients
print(model_linear.summary())

In [ ]:
# Extract p-values
p_values = model_linear.pvalues[1:] # exclude the intercept

# Apply Bonferroni correction
alpha = 0.05
reject, adjusted_p_values, _, _ = multipletests(p_values, alpha=alpha, method='bonferroni')

# Print the results
print('P-values:', p_values)
print('Adjusted p-values:', adjusted_p_values)
print('Rejected hypotheses:', reject)

In [ ]:
# Run Breusch-Pagan test for heteroskedasticity
bp_test = het_breuschpagan(model_linear.resid, model_linear.model.exog)
print('Breusch-Pagan test p-value:', bp_test[1]) # heteroskedastic

# Run Durbin-Watson test for autocorrelation
dw_test = durbin_watson(model_linear.resid)
print('Durbin-Watson test statistic:', dw_test)
print('Durbin-Watson test p-value:', 2 * norm.cdf(-abs(dw_test - 2))) # no autocorrelation

In [ ]:
# Prepare test data set
test_X = test_X.drop(["GDP growth", "Consumption of durables", "Inflation", "Quarter_2", "Exchange rate", "Unemployment rate", "Quarter_3", "Consumer confidence index", "Interest rate", "Age"], axis = "columns")
test_X = sm.add_constant(test_X)
# Calculate perdictions for test data set
pred = model_linear.predict(test_X)

In [ ]:
# Calculate MAE
print(f"MAE: {mean_absolute_error(test_y, pred):.4f}")

In [ ]:
# Calculate MSE 
print(f"MSE: {mean_squared_error(test_y, pred):.4f}")

In [ ]:
# Calculate RMSE
print(f"RMSE: {mean_squared_error(test_y, pred, squared=False):.4f}")

# Neural nets

## Random search with neural nets

### Data recurring

In [ ]:
# Upload data
with open('Data final//data_recurring_imputed.pkl', 'rb') as file:
    data = pickle.load(file)

In [ ]:
# Sample 1000000 rows to perform a random search
index_sample = random.sample(range(0, len(data)), 1000000)
data = data.iloc[index_sample,]
# Divide data into dependent and independent variables
X = data.drop(["Total_revenue"], axis = "columns")
y = data["Total_revenue"]
# Divide data into train and test data sets
train_X, test_X, train_y, test_y = train_test_split(X, y, test_size = 0.25, random_state = 3)
# Standardise data 
scaler = StandardScaler()
train_X = scaler.fit_transform(train_X)
test_X = scaler.transform(test_X)
# Choose ten learning rates ranging from 0.000001 to 0.001
learning_rates = list(np.logspace(-6, -3, 10))

In [ ]:
# Define the architecture of a neural network 
def create_model_neural_net(hp):
    model_net = Sequential()
    # Add a hidden layer with 2000 hidden units and a ReLu activation function
    model_net.add(Dense(2000, input_shape=(train_X.shape[1],), activation='relu'))
    # Add a dropout layer with a 50% dropout rate
    model_net.add(Dropout(0.5))
    # Add a hidden layer with 1000 hidden units and a ReLu activation function
    model_net.add(Dense(1000, activation='relu'))
    # Add a dropout layer with a 30% dropout rate
    model_net.add(Dropout(0.3))
    # Add a hidden layer with 500 hidden units and a ReLu activation function
    model_net.add(Dense(500, activation='relu'))
    # Add a dropout layer with a 30% dropout rate
    model_net.add(Dropout(0.3))
     # Add a hidden layer with 250 hidden units and a ReLu activation function
    model_net.add(Dense(250, activation='relu'))
    # Add a dropout layer with a 20% dropout rate
    model_net.add(Dropout(0.2))
     # Add an output layer with 1 hidden unit and a linear activation function
    model_net.add(Dense(1, activation='linear'))
    
    # Choose a loss function, evaluation metrics and optimizers
    model_net.compile(optimizer=hp.Choice('optimizer', values=['adam', 'rmsprop']), 
                  loss='mse', 
                  metrics=['mae'])

    # Set possible learning rates that will be checked
    model_net.optimizer.learning_rate = hp.Choice('learning_rate', values=[1e-06, 2.1544346900318822e-06, 4.641588833612782e-06, 1e-05, 2.1544346900318823e-05, 4.641588833612772e-05, 0.0001, 0.00021544346900318823, 0.00046415888336127773, 0.001])
    return model_net

# Define the search space of hyperparameters
tuner_hp = HyperParameters()
# Set possible optimizers that will be checked
tuner_hp.Choice('optimizer', values=['adam', "rmsprop"])
# Set possible learning rates that will be checked
tuner_hp.Choice('learning_rate', values= [1e-06, 2.1544346900318822e-06, 4.641588833612782e-06, 1e-05, 2.1544346900318823e-05, 4.641588833612772e-05, 0.0001, 0.00021544346900318823, 0.00046415888336127773, 0.001])

# Create the tuner object
tuner = RandomSearch(
    # Assign a neural network 
    create_model_neural_net,
    # Assign available parameters
    hyperparameters=tuner_hp,
    # Choose validation loss as the optimisation objective
    objective='val_loss',
    # Choose the maximum number of trials to check
    max_trials=10,
    overwrite = True,
    project_name='my_project')

# Set an early stopping procedure
early_stopping = EarlyStopping(
    # Choose validation loss as the monitored metric
    monitor='val_loss', 
    # Set the patience to 5. Patience is the number of epochs with no improvement after which training will be stopped
    patience = 5, 
    # Set the mode to 'min', which means that training will stop when the quantity monitored has stopped decreasing
    mode = "min", 
    # Leave the weights which generated the lowest validation loss
    restore_best_weights = True,
    # Set the minimum change in the monitored quantity to qualify as an improvement to 0.01
    min_delta=0.01
    )

# Run the search
tuner.search(train_X, 
             train_y,
             # Run the search for 30 epochs
             epochs=30, 
             # Split the data such that 80% is allocated for training and 20% is reserved for validation
             validation_split = 0.2,
             callbacks=[early_stopping]
             )

In [ ]:
# Extract the best model
model_neural_net_best=tuner.get_best_models()[0]

In [ ]:
# Extract the parameters of the best model
best_hyperparameters = tuner.get_best_hyperparameters(num_trials=1)[0]
learning_rate = best_hyperparameters.get('learning_rate')
optimizer = best_hyperparameters.get('optimizer')
print(f"Best learning rate: {learning_rate}")
print(f"Best optimizer: {optimizer}")

In [ ]:
# Evaluate the best model based on MAE
model_neural_net_best.evaluate(test_X, test_y)

### Data new

In [ ]:
# Upload data
with open('Data final//data_new_imputed.pkl', 'rb') as file:
    data = pickle.load(file)

In [ ]:
# Divide data into dependent and independent variables
X = data.drop(["Total_revenue"], axis = "columns")
y = data["Total_revenue"]
# Divide data into train and test data sets
train_X, test_X, train_y, test_y = train_test_split(X, y, test_size = 0.25, random_state = 3)
# Standardise data 
scaler = StandardScaler()
train_X = scaler.fit_transform(train_X)
test_X = scaler.transform(test_X)
# Choose ten learning rates ranging from 0.000001 to 0.001
learning_rates = list(np.logspace(-6, -3, 10))

In [ ]:
# Define the architecture of a neural network 
def create_model_neural_net(hp):
    model_net = Sequential()
    # Add a hidden layer with 2000 hidden units and a ReLu activation function
    model_net.add(Dense(2000, input_shape=(train_X.shape[1],), activation='relu'))
    # Add a dropout layer with a 50% dropout rate
    model_net.add(Dropout(0.5))
    # Add a hidden layer with 1000 hidden units and a ReLu activation function
    model_net.add(Dense(1000, activation='relu'))
    # Add a dropout layer with a 30% dropout rate
    model_net.add(Dropout(0.3))
    # Add a hidden layer with 500 hidden units and a ReLu activation function
    model_net.add(Dense(500, activation='relu'))
    # Add a dropout layer with a 30% dropout rate
    model_net.add(Dropout(0.3))
     # Add a hidden layer with 250 hidden units and a ReLu activation function
    model_net.add(Dense(250, activation='relu'))
    # Add a dropout layer with a 20% dropout rate
    model_net.add(Dropout(0.2))
     # Add an output layer with 1 hidden unit and a linear activation function
    model_net.add(Dense(1, activation='linear'))
    
    # Choose a loss function, evaluation metrics and optimizers
    model_net.compile(optimizer=hp.Choice('optimizer', values=['adam', 'rmsprop']), 
                  loss='mse', 
                  metrics=['mae'])

    # Set possible learning rates that will be checked
    model_net.optimizer.learning_rate = hp.Choice('learning_rate', values=[1e-06, 2.1544346900318822e-06, 4.641588833612782e-06, 1e-05, 2.1544346900318823e-05, 4.641588833612772e-05, 0.0001, 0.00021544346900318823, 0.00046415888336127773, 0.001])
    return model_net

# Define the search space of hyperparameters
tuner_hp = HyperParameters()
# Set possible optimizers that will be checked
tuner_hp.Choice('optimizer', values=['adam', "rmsprop"])
# Set possible learning rates that will be checked
tuner_hp.Choice('learning_rate', values= [1e-06, 2.1544346900318822e-06, 4.641588833612782e-06, 1e-05, 2.1544346900318823e-05, 4.641588833612772e-05, 0.0001, 0.00021544346900318823, 0.00046415888336127773, 0.001])

# Create the tuner object
tuner = RandomSearch(
    # Assign a neural network 
    create_model_neural_net,
    # Assign available parameters
    hyperparameters=tuner_hp,
    # Choose validation loss as the optimisation objective
    objective='val_loss',
    # Choose the maximum number of trials to check
    max_trials=10,
    overwrite = True,
    project_name='my_project')

# Set an early stopping procedure
early_stopping = EarlyStopping(
    # Choose validation loss as the monitored metric
    monitor='val_loss', 
    # Set the patience to 5. Patience is the number of epochs with no improvement after which training will be stopped
    patience = 5, 
    # Set the mode to 'min', which means that training will stop when the quantity monitored has stopped decreasing
    mode = "min", 
    # Leave the weights which generated the lowest validation loss
    restore_best_weights = True,
    # Set the minimum change in the monitored quantity to qualify as an improvement to 0.01
    min_delta=0.01
    )

# Run the search
tuner.search(train_X, 
             train_y,
             # Run the search for 30 epochs
             epochs=30, 
             # Split the data such that 80% is allocated for training and 20% is reserved for validation
             validation_split = 0.2,
             callbacks=[early_stopping]
             )

In [ ]:
# Extract the best model
model_neural_net_best=tuner.get_best_models()[0]

In [ ]:
# Extract the parameters of the best model
best_hyperparameters = tuner.get_best_hyperparameters(num_trials=1)[0]
learning_rate = best_hyperparameters.get('learning_rate')
optimizer = best_hyperparameters.get('optimizer')
print(f"Best learning rate: {learning_rate}")
print(f"Best optimizer: {optimizer}")

In [ ]:
# Evaluate the best model based on MAE
model_neural_net_best.evaluate(test_X, test_y)

## Model neural nets with the best parameters

### Data recurring

In [ ]:
# Upload data
with open('Data final//data_recurring_imputed.pkl', 'rb') as file:
    data = pickle.load(file)

In [ ]:
# Sample 1000000 observations from data
index_sample = random.sample(range(0, len(data)), 1000000)
data = data.iloc[index_sample,]

In [ ]:
# Divide data into dependent and independent variables
X = data.drop(["Total_revenue"], axis = "columns")
y = data["Total_revenue"]
# Divide data into train and test data sets
train_X, test_X, train_y, test_y = train_test_split(X, y, test_size = 0.25, random_state = 3)
# Standardise data 
scaler = StandardScaler()
train_X = scaler.fit_transform(train_X)
test_X = scaler.transform(test_X)

In [ ]:
# Define the architecture of a neural network 
model_net = Sequential()
# Add a hidden layer with 2000 hidden units and a ReLu activation function
model_net.add(Dense(2000, input_shape=(train_X.shape[1],), activation='relu'))
# Add a dropout layer with a 50% dropout rate
model_net.add(Dropout(0.5))
# Add a hidden layer with 1000 hidden units and a ReLu activation function
model_net.add(Dense(1000, activation='relu'))
# Add a dropout layer with a 30% dropout rate
model_net.add(Dropout(0.3))
# Add a hidden layer with 500 hidden units and a ReLu activation function
model_net.add(Dense(500, activation='relu'))
# Add a dropout layer with a 30% dropout rate
model_net.add(Dropout(0.3))
# Add a hidden layer with 250 hidden units and a ReLu activation function
model_net.add(Dense(250, activation='relu'))
# Add a dropout layer with a 20% dropout rate
model_net.add(Dropout(0.2))
# Add an output layer with 1 hidden unit and a linear activation function
model_net.add(Dense(1, activation='linear'))

# Compile the neural network with an Adam optimizer and a learning rate of 0.0001
model_net.compile(loss='mse', optimizer= Adam(learning_rate=0.0001), metrics=['mae'])

# Define the checkpoint path and a metric to monitor
checkpoint_path = "best_model.h5"
monitor_metric = 'val_loss'

# Create the ModelCheckpoint callback
checkpoint = ModelCheckpoint(checkpoint_path, monitor=monitor_metric, save_best_only=True, mode='max')

# Set an early stopping procedure
es = EarlyStopping(
    # Choose validation loss as the monitored metric
    monitor='val_loss',
    # Set the mode to 'min', which means that training will stop when the quantity monitored has stopped decreasing
    mode='min',
    # Set the patience to 5. Patience is the number of epochs with no improvement after which training will be stopped.
    patience=5,
    # Leave the weights which generated the lowest validation loss
    restore_best_weights = True)

# Fit the model 
history  = model_net.fit(train_X, train_y, 
                    # Run the search for 30 epochs
                    epochs=30, 
                    # Choose the batch sie
                    batch_size=32,
                    # Split the data such that 80% is allocated for training and 20% is reserved for validation.
                    validation_split = 0.2,
                    callbacks=[es, checkpoint])

In [ ]:
# Evaluate the model  
model_net.evaluate(test_X, test_y)
# Calculate predictions
pred = model_net.predict(test_X)

In [ ]:
# Calculate MSE 
mean_squared_error(test_y, pred)
# Calculate RMSE
mean_squared_error(test_y, pred, squared=False)

### Data new

In [ ]:
# Upload data
with open('Data final//data_new_imputed.pkl', 'rb') as file:
    data = pickle.load(file)

In [ ]:
# Divide data into dependent and independent variables
X = data.drop(["Total_revenue"], axis = "columns")
y = data["Total_revenue"]
# Divide data into train and test data sets
train_X, test_X, train_y, test_y = train_test_split(X, y, test_size = 0.25, random_state = 3)
# Standardise data 
scaler = StandardScaler()
train_X = scaler.fit_transform(train_X)
test_X = scaler.transform(test_X)

In [ ]:
# Define the architecture of a neural network 
model_net = Sequential()
# Add a hidden layer with 2000 hidden units and a ReLu activation function
model_net.add(Dense(2000, input_shape=(train_X.shape[1],), activation='relu'))
# Add a dropout layer with a 50% dropout rate
model_net.add(Dropout(0.5))
# Add a hidden layer with 1000 hidden units and a ReLu activation function
model_net.add(Dense(1000, activation='relu'))
# Add a dropout layer with a 30% dropout rate
model_net.add(Dropout(0.3))
# Add a hidden layer with 500 hidden units and a ReLu activation function
model_net.add(Dense(500, activation='relu'))
# Add a dropout layer with a 30% dropout rate
model_net.add(Dropout(0.3))
# Add a hidden layer with 250 hidden units and a ReLu activation function
model_net.add(Dense(250, activation='relu'))
# Add a dropout layer with a 20% dropout rate
model_net.add(Dropout(0.2))
# Add an output layer with 1 hidden unit and a linear activation function
model_net.add(Dense(1, activation='linear'))

# Compile the neural network with an Adam optimizer and a learning rate of 0.0001
model_net.compile(loss='mse', optimizer= Adam(learning_rate=0.00046), metrics=['mae'])

# Define the checkpoint path and a metric to monitor
checkpoint_path = "best_model.h5"
monitor_metric = 'val_loss'

# Create the ModelCheckpoint callback
checkpoint = ModelCheckpoint(checkpoint_path, monitor=monitor_metric, save_best_only=True, mode='max')

# Set an early stopping procedure
es = EarlyStopping(
    # Choose validation loss as the monitored metric
    monitor='val_loss',
    # Set the mode to 'min', which means that training will stop when the quantity monitored has stopped decreasing
    mode='min',
    # Set the patience to 5. Patience is the number of epochs with no improvement after which training will be stopped.
    patience=5,
    # Leave the weights which generated the lowest validation loss
    restore_best_weights = True)

# Fit the model 
history  = model_net.fit(train_X, train_y, 
                    # Run the search for 30 epochs
                    epochs=30, 
                    # Choose the batch sie
                    batch_size=32,
                    # Split the data such that 80% is allocated for training and 20% is reserved for validation.
                    validation_split = 0.2,
                    callbacks=[es, checkpoint])

In [ ]:
# Evaluate the model  
model_net.evaluate(test_X, test_y)
# Calculate predictions
pred = model_net.predict(test_X)

In [ ]:
# Calculate MSE 
mean_squared_error(test_y, pred)
# Calculate RMSE
mean_squared_error(test_y, pred, squared=False)

# XGBoost

## Random search with XGBoost

### Data recurring

In [ ]:
# Upload data
with open('Data final//data_recurring_imputed.pkl', 'rb') as file:
    data = pickle.load(file)

In [ ]:
# Sample 1000000 rows to perform a random search
index_sample = random.sample(range(0, len(data)), 1000000)
data = data.iloc[index_sample,]

In [ ]:
# Divide data into dependent and independent variables
X = data.drop(["Total_revenue"], axis = "columns")
y = data["Total_revenue"]
# Divide data into train and test data sets
train_X, test_X, train_y, test_y = train_test_split(X, y, test_size = 0.25, random_state = 3)

In [ ]:
# Set parameters that will be checked for a random search
params = {
    'n_estimators':[70, 100, 200],
    'min_child_weight':[1, 2, 3], 
    'gamma':[i/10.0 for i in range(0,3)],  
    'subsample':[i/10.0 for i in range(6,11)],
    'colsample_bytree':[i/10.0 for i in range(6,11)], 
    'max_depth': [2,3,4,6,7],
    'objective': ['reg:squarederror'],
    'booster': ['gbtree', 'gblinear'],
    'eta': [i/10.0 for i in range(3,6)],
}

# Define a model
reg = XGBRegressor(nthread=-1)

# Set the number of iterations for a random search
n_iter_search = 50
# Define a random search
random_search = RandomizedSearchCV(reg, param_distributions=params,
                                   n_iter=n_iter_search, cv=5, scoring='neg_mean_absolute_error')


# Perform a random search
random_search.fit(train_X, train_y)

In [ ]:
# Extract the best model
best_regressor_xgb = random_search.best_estimator_
# Extract the best parameters
random_search.best_params_

In [ ]:
# Get predictions
pred = best_regressor_xgb.predict(test_X)
# Calculate MAE
mean_absolute_error(test_y, pred) 

In [ ]:
# Calculate RMSE
mean_squared_error(test_y, pred, squared=False)

### Data new

In [ ]:
# Upload data
with open('Data final//data_new_imputed.pkl', 'rb') as file:
    data = pickle.load(file)

In [ ]:
# Divide data into dependent and independent variables
X = data.drop(["Total_revenue"], axis = "columns")
y = data["Total_revenue"]
# Divide data into train and test data sets
train_X, test_X, train_y, test_y = train_test_split(X, y, test_size = 0.25, random_state = 3)

In [ ]:
# Set parameters that will be checked for a random search
params = {
    'n_estimators':[70, 100, 200],
    'min_child_weight':[1, 2, 3], 
    'gamma':[i/10.0 for i in range(0,3)],  
    'subsample':[i/10.0 for i in range(6,11)],
    'colsample_bytree':[i/10.0 for i in range(6,11)], 
    'max_depth': [2,3,4,6,7],
    'objective': ['reg:squarederror'],
    'booster': ['gbtree', 'gblinear'],
    'eta': [i/10.0 for i in range(3,6)],
}

# Define a model
reg = XGBRegressor(nthread=-1)

# Set the number of iterations for a random search
n_iter_search = 50
# Define a random search
random_search = RandomizedSearchCV(reg, param_distributions=params,
                                   n_iter=n_iter_search, cv=5, scoring='neg_mean_absolute_error')


# Perform a random search
random_search.fit(train_X, train_y)

In [ ]:
# Extract the best model
best_regressor_xgb = random_search.best_estimator_
# Extract the best parameters
random_search.best_params_

In [ ]:
# Get predictions
pred = best_regressor_xgb.predict(test_X)
# Calculate MAE
mean_absolute_error(test_y, pred) 

In [ ]:
# Calculate RMSE
mean_squared_error(test_y, pred, squared=False)

## Model XGBoost with the best parameters

### Data recurring

In [ ]:
# Upload data
with open('Data final//data_recurring_imputed.pkl', 'rb') as file:
    data = pickle.load(file)

In [ ]:
# Divide data into dependent and independent variables
X = data.drop(["Total_revenue"], axis = "columns")
y = data["Total_revenue"]
# Divide data into train and test data sets
train_X, test_X, train_y, test_y = train_test_split(X, y, test_size = 0.25, random_state = 3)

In [ ]:
# Define a model with the best parameters 
model_xgb = XGBRegressor(subsample = 0.9, objective = 'reg:squarederror', n_estimators = 200, min_child_weight = 2, 
                    max_depth = 6, gamma = 0.1, eta = 0.3, colsample_bytree = 0.8, booster = 'gbtree')
  
# Fit a model
model_xgb.fit(train_X, train_y)

In [ ]:
# Calculate predictions
pred = model_xgb.predict(test_X)

In [ ]:
# Calculate MAE
mean_absolute_error(test_y, pred) 

In [ ]:
# Calculate MSE
mean_squared_error(test_y, pred)

In [ ]:
# Calculate RMSE
mean_squared_error(test_y, pred, squared=False)

#### Variable importance with Shapley values

In [ ]:
# Sample 20 000 observations to calculate Shapley values
index_sample = random.sample(range(0, len(test_X)), 20000)
shap_X = test_X.iloc[index_sample]

# Get Shapley values
shap_values = shap.TreeExplainer(model_xgb).shap_values(shap_X)

In [ ]:
# Plot the variables by importance (box plot)
fig = plt.figure()
shap.summary_plot(shap_values, shap_X, plot_type="bar", show = False, color = "#9797ff")
plt.xlabel("Shapley value")
plt.xlim([0,300])
plt.show()

In [ ]:
# Plot the variables with detail of Shapley values (beeswarm plot)
fig = plt.figure()
shap.summary_plot(shap_values, shap_X, show = False, cmap = "coolwarm")
plt.xlabel("Shapley value")
plt.xlim([-2000,5500])
plt.show()

### Data new

In [ ]:
# Upload data
with open('Data final//data_new_imputed.pkl', 'rb') as file:
    data = pickle.load(file)

In [ ]:
# Divide data into dependent and independent variables
X = data.drop(["Total_revenue"], axis = "columns")
y = data["Total_revenue"]
# Divide data into train and test data sets
train_X, test_X, train_y, test_y = train_test_split(X, y, test_size = 0.25, random_state = 3)

In [ ]:
# Define a model
model_xgb = XGBRegressor(subsample = 1, objective = 'reg:squarederror', n_estimators = 70, min_child_weight = 3, 
                    max_depth = 6, gamma = 0.2, eta = 0.3, colsample_bytree = 0.8, booster = 'gbtree')
  
# Fit a model
model_xgb.fit(train_X, train_y)

In [ ]:
# Calculate predictions
pred = model_xgb.predict(test_X)

In [ ]:
# Calculate MAE
mean_absolute_error(test_y, pred) 

In [ ]:
# Calculate MSE
mean_squared_error(test_y, pred)

In [ ]:
# Calculate RMSE
mean_squared_error(test_y, pred, squared=False)

#### Variable importance with Shapley values

In [ ]:
# Sample 20 000 observations to calculate Shapley values
index_sample = random.sample(range(0, len(test_X)), 20000)
shap_X = test_X.iloc[index_sample]

# Get Shapley values
shap_values = shap.TreeExplainer(model_xgb).shap_values(shap_X)

In [ ]:
# Plot the variables by importance (box plot)
fig = plt.figure()
shap.summary_plot(shap_values, shap_X, plot_type="bar", show = False, color = "#9797ff")
plt.xlabel("Shapley value")
plt.xlim([0,60])
plt.show()

In [ ]:
# Plot the variables with detail of Shapley values (beeswarm plot)
fig = plt.figure()
shap.summary_plot(shap_values, shap_X, show = False, cmap = "coolwarm")
plt.xlabel("Shapley value")
plt.xlim([-1500,3000])
plt.show()

# Random forest

## Random search with Random forest

### Data recurring

In [ ]:
# Upload data
with open('Data final//data_recurring_imputed.pkl', 'rb') as file:
    data = pickle.load(file)

In [ ]:
# Sample 50000 observations from data
index_sample = random.sample(range(0, len(data)), 50000)
data = data.iloc[index_sample,]

In [ ]:
# Divide data into dependent and independent variables
y = data["Total_revenue"]
X = data.drop("Total_revenue", axis = "columns")

In [ ]:
# Define search range for random search for...
# number of decision trees to use
n_estimators = [40, 50, 60, 70, 80] 
# loss function
criterion = ["squared_error", "absolute_error"]
# maximum number of features to consider at every split
max_features = ["log2", "sqrt"]
# maximum number of levels in each tree
max_depth = [3, 5, 10, 15, 20]
# minimum number of observations to split a node 
min_samples_split = [0.0005, 0.001, 0.0025, 0.005] 
# use bootstrap samples
bootstrap = [True]

In [ ]:
# Create a random grid
random_grid = {"n_estimators": n_estimators, "criterion": criterion, "max_features": max_features, "max_depth": max_depth,
               "min_samples_split": min_samples_split, "bootstrap": bootstrap}
# Define estimator (random forest)
rf = RandomForestRegressor()

In [ ]:
# Random search (tries 50 different random combinations out of 800 possible ones)
rf_random_search = RandomizedSearchCV(estimator = rf, param_distributions = random_grid,
                                      n_iter = 50, cv = 2, verbose = 2, n_jobs = -1)
rf_random_search_fit = rf_random_search.fit(X, y)

In [ ]:
# Extract the best parameters found in the random search 
rf_random_search_best = rf_random_search.best_params_ 
print(rf_random_search_best)

### Data new

In [ ]:
# Upload data
with open('Data final//data_new_imputed.pkl', 'rb') as file:
    data = pickle.load(file)

In [ ]:
# Sample 50000 rows to perform a random search
index_sample = random.sample(range(0, len(data)), 50000)
data = data.iloc[index_sample,]

In [ ]:
# Divide data into dependent and independent variables
y = data["Total_revenue"]
X = data.drop("Total_revenue", axis = "columns")

In [ ]:
# Define search range for random search for...
# number of decision trees to use
n_estimators = [40, 50, 60, 70, 80] 
# loss function
criterion = ["squared_error", "absolute_error"]
# maximum number of features to consider at every split
max_features = ["log2", "sqrt"]
# maximum number of levels in each tree
max_depth = [3, 5, 10, 15, 20]
# minimum number of observations to split a node 
min_samples_split = [0.0005, 0.001, 0.0025, 0.005] 
# use bootstrap samples
bootstrap = [True]

In [ ]:
# Create a random grid
random_grid = {"n_estimators": n_estimators, "criterion": criterion, "max_features": max_features, "max_depth": max_depth,
               "min_samples_split": min_samples_split, "bootstrap": bootstrap}
# Define estimator (random forest)
rf = RandomForestRegressor()

In [ ]:
# Random search (tries 50 different random combinations out of 800 possible ones)
rf_random_search = RandomizedSearchCV(estimator = rf, param_distributions = random_grid,
                                      n_iter = 50, cv = 2, verbose = 2, n_jobs = -1)
rf_random_search_fit = rf_random_search.fit(X, y)

In [ ]:
# Extract the best parameters found in the random search 
rf_random_search_best = rf_random_search.best_params_ 
print(rf_random_search_best)

## Model Random forest with the best parameters

### Data recurring

In [ ]:
# Upload data
with open('Data final//data_recurring_imputed.pkl', 'rb') as file:
    data = pickle.load(file)

In [ ]:
# Divide data into dependent and independent variables
y = data["Total_revenue"]
X = data.drop("Total_revenue", axis = "columns")

In [ ]:
# Divide data into train and test data sets
train_X, test_X, train_y, test_y = train_test_split(X, y, test_size = 0.25)

In [ ]:
# Instantiate model with optimal parameters
rf = RandomForestRegressor(n_estimators = 60, min_samples_split = 0.0005, max_features = "sqrt",
                          max_depth = 15, criterion = "squared_error", bootstrap = False)

# Train the model on training data
rf.fit(train_X, train_y)

In [ ]:
# Get predictions for the test dataset
rf_pred = rf.predict(test_X)

In [ ]:
# Calculate test error (mean absolute error)
rf_error = abs(rf_pred - test_y)
rf_mae = np.mean(rf_error)
# Calculate test error (mean squared error)
rf_mse = mean_squared_error(test_y, rf_pred)
# Calculate test error (root mean squared error)
rf_rmse = mean_squared_error(test_y, rf_pred, squared=False)

In [ ]:
rf_mae

In [ ]:
rf_mse

In [ ]:
rf_rmse

#### Variable importance with Shapley values

In [ ]:
# Sample 20 000 observations to calculate Shapley values
index_sample = random.sample(range(0, len(test_X)), 20000)
shap_X = test_X.iloc[index_sample]

# Get Shapley values
shap_values = shap.TreeExplainer(rf).shap_values(shap_X)

In [ ]:
# Plot the variables by importance (box plot)
fig = plt.figure()
shap.summary_plot(shap_values, shap_X, plot_type="bar", show = False, color = "#9797ff")
plt.xlabel("Shapley value")
plt.xlim([0,160])
plt.show()

In [ ]:
# Plot the variables with detail of Shapley values (beeswarm plot)
fig = plt.figure()
shap.summary_plot(shap_values, shap_X, show = False, cmap = "coolwarm")
plt.xlabel("Shapley value")
plt.xlim([-700,3500])
plt.show()

### Data new

In [ ]:
# Upload data
with open('Data final//data_new_imputed.pkl', 'rb') as file:
    data = pickle.load(file)

In [ ]:
# Divide data into dependent and independent variables
y = data["Total_revenue"]
X = data.drop("Total_revenue", axis = "columns")

In [ ]:
# Divide data into train and test data sets
train_X, test_X, train_y, test_y = train_test_split(X, y, test_size = 0.25)

In [ ]:
# Instantiate model with optimal parameters
rf = RandomForestRegressor(n_estimators = 80, min_samples_split = 0.001, max_features = "log2",
                          max_depth = 20, criterion = "squared_error", bootstrap = True)
# Train the model on training data
rf.fit(train_X, train_y)

In [ ]:
# Get predictions for the test dataset
rf_pred = rf.predict(test_X)
# Calculate test error (mean absolute error)
rf_error = abs(rf_pred - test_y)
rf_mae = np.mean(rf_error)
# Calculate test error (mean squared error)
rf_mse = mean_squared_error(test_y, rf_pred)
# Calculate test error (root mean squared error)
rf_rmse = mean_squared_error(test_y, rf_pred, squared=False)

In [ ]:
rf_mae

In [ ]:
rf_mse

In [ ]:
rf_rmse

#### Variable importance with Shapley values

In [ ]:
# Sample 20 000 observations to calculate Shapley values
index_sample = random.sample(range(0, len(test_X)), 20000)
shap_X = test_X.iloc[index_sample]

# Get Shapley values
shap_values = shap.TreeExplainer(rf).shap_values(shap_X)

In [ ]:
# Plot the variables by importance (box plot)
fig = plt.figure()
shap.summary_plot(shap_values, shap_X, plot_type="bar", show = False, color = "#9797ff")
plt.xlabel("Shapley value")
plt.xlim([0,160])
plt.show()

In [ ]:
# Plot the variables with detail of Shapley values (beeswarm plot)
fig = plt.figure()
shap.summary_plot(shap_values, shap_X, show = False, cmap = "coolwarm")
plt.xlabel("Shapley value")
plt.xlim([-700,3500])
plt.show()

# Support vector regression

## Random search with Support vector regression

### Data recurring

In [ ]:
# Upload data
with open('Data final//data_recurring_imputed.pkl', 'rb') as file:
    data = pickle.load(file)

In [ ]:
# Sample 40000 observations from data
index_sample = random.sample(range(0, len(data)), 40000)
data = data.iloc[index_sample,]

In [ ]:
# Dataframe with features and array with outcomes (shaped in 2d array)
y = data["Total revenue"].values
X = data.drop("Total revenue", axis = "columns").values
y = y.reshape(-1,1)

In [ ]:
# Rescale and standardize 
StdS_X = RobustScaler()
StdS_y = RobustScaler()
X = StdS_X.fit_transform(X)
y = StdS_y.fit_transform(y)

In [ ]:
# Sample splitting
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25)

In [ ]:
# Transform a vector
y_train = np.ravel(y_train)

In [ ]:
# List of C values
C_range = np.logspace(-5, 5, 10)
print(f'The list of values for C are {C_range}')
# List of gamma values
gamma_range = np.logspace(-5, 5, 10)
print(f'The list of values for gamma are {gamma_range}')

In [ ]:
# Define the search space
param_grid = { 
    # Regularization parameter.
    "C": C_range,
    # Kernel type
    "kernel": ['rbf'],
    # Gamma is the Kernel coefficient for ‘rbf’, ‘poly’ and ‘sigmoid’
    "gamma": gamma_range.tolist()+['scale', 'auto']
    }
# Set up score
mae_scorer = make_scorer(mean_absolute_error, greater_is_better=False)

#scoring = ['neg_mean_absolute_error'] # uses MAE as the metric

# Set up the k-fold cross-validation
kfold = KFold(n_splits=3, shuffle=True, random_state=0) # shuffle - data is shuffled before splitting
# random_state = 0 makes the shuffle reproducible
# n_splits = 3 - 3-fold cross-validation

# Define random search
random_search = RandomizedSearchCV(estimator=SVR(), # chooses SVR
                           param_distributions=param_grid, # takes our pre-defined search space for the grid search
                           n_iter=10, # the number of parameter combinations sampled
                           scoring= mae_scorer, # set the performance evaluation metric
                           refit='neg_mean_absolute_error', # enables refitting the model with the best parameters on the whole training dataset.
                           n_jobs=-1, # means parallel processing using all the processors
                           cv=kfold, # takes the StratifiedKFold we defined
                           verbose=0) # controls the number of messages returned by random search
# Fit grid search
random_result = random_search.fit(X_train, y_train)
# Print grid search summary
print(random_result)

In [ ]:
# Print the best accuracy score for the training dataset
print(f'The best MAE score for the training dataset is {random_result.best_score_:.4f}')
# Print the hyperparameters for the best score
print(f'The best hyperparameters are {random_result.best_params_}')
# Print the best accuracy score for the testing dataset
print(f'The MAE score for the testing dataset is {random_search.score(X_test, y_test):.4f}')

In [ ]:
best_svr_model = random_search.best_estimator_

# Calculate predictions for test data
predictions = StdS_y.inverse_transform(best_svr_model.predict(X_test).reshape(-1,1))

In [ ]:
# Calculate MAE
print(f"MAE: {mean_absolute_error(StdS_y.inverse_transform(y_test), predictions):.4f}")

In [ ]:
# Calculate MSE
print(f"MAE: {mean_squared_error(StdS_y.inverse_transform(y_test), predictions):.4f}")

In [ ]:
print(f"MAE: {mean_squared_error(StdS_y.inverse_transform(y_test), predictions, squared =False):.4f}")

### Data new

In [ ]:
# Upload data
with open('Data final//data_new_imputed.pkl', 'rb') as file:
    data = pickle.load(file)

In [ ]:
# Sample 40000 observations from data
index_sample = random.sample(range(0, len(data)), 40000)
data = data.iloc[index_sample,]

In [ ]:
# Dataframe with features and array with outcomes (shaped in 2d array)
y = data["Total_revenue"].values
X = data.drop("Total_revenue", axis = "columns").values
y = y.reshape(-1,1)

In [ ]:
# Rescale and standardize 
StdS_X = RobustScaler()
StdS_y = RobustScaler()
X = StdS_X.fit_transform(X)
y = StdS_y.fit_transform(y)

In [ ]:
# Sample splitting
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25)

In [ ]:
# Transform a vector
y_train = np.ravel(y_train)

In [ ]:
# Define the search space
param_grid = { 
    # Regularization parameter.
    "C": C_range,
    # Kernel type
    "kernel": ['rbf'],
    # Gamma is the Kernel coefficient for ‘rbf’, ‘poly’ and ‘sigmoid’.
    "gamma": gamma_range.tolist()+['scale', 'auto']
    }
# Set up score
mae_scorer = make_scorer(mean_absolute_error, greater_is_better=False)

#scoring = ['neg_mean_absolute_error'] # uses MAE as the metric

# Set up the k-fold cross-validation
kfold = KFold(n_splits=3, shuffle=True, random_state=0) # shuffle - data is shuffled before splitting
# random_state = 0 makes the shuffle reproducible
# n_splits = 3 - 3-fold cross-validation

# Define random search
random_search = RandomizedSearchCV(estimator=SVR(), # chooses SVR
                           param_distributions=param_grid, # takes our pre-defined search space for the grid search
                           n_iter=10, # the number of parameter combinations sampled
                           scoring= mae_scorer, # set the performance evaluation metric
                           refit='neg_mean_absolute_error', # enables refitting the model with the best parameters on the whole training dataset.
                           n_jobs=-1, # means parallel processing using all the processors
                           cv=kfold, # takes the StratifiedKFold we defined
                           verbose=0) # controls the number of messages returned by random search
# Fit grid search
random_result = random_search.fit(X_train, y_train)
# Print grid search summary
print(random_result)

In [ ]:
# Print the best accuracy score for the training dataset
print(f'The best MAE score for the training dataset is {random_result.best_score_:.4f}')
# Print the hyperparameters for the best score
print(f'The best hyperparameters are {random_result.best_params_}')
# Print the best accuracy score for the testing dataset
print(f'The MAE score for the testing dataset is {random_search.score(X_test, y_test):.4f}')

In [ ]:
best_svr_model = random_search.best_estimator_
# Calculate predictions for test data
predictions = StdS_y.inverse_transform(best_svr_model.predict(X_test).reshape(-1,1))

In [ ]:
# Calculate MAE
print(f"MAE: {mean_absolute_error(StdS_y.inverse_transform(y_test), predictions):.4f}")

In [ ]:
# Calculate MSE
print(f"MAE: {mean_squared_error(StdS_y.inverse_transform(y_test), predictions):.4f}")

In [ ]:
print(f"MAE: {mean_squared_error(StdS_y.inverse_transform(y_test), predictions, squared =False):.4f}")

## Model Support Vector regression with the best parameters

### Data recurring

In [ ]:
# Upload data
with open('Data final//data_recurring_imputed.pkl', 'rb') as file:
    data = pickle.load(file)

In [ ]:
# Sample 100000 observations from data
index_sample = random.sample(range(0, len(data)), 100000)
data = data.iloc[index_sample,]
# Dataframe with features and array with outcomes (shaped in 2d array)
y = data["Total revenue"].values
X = data.drop("Total revenue", axis = "columns").values
y = y.reshape(-1,1)
# Rescale and standardize 
StdS_X = RobustScaler()
StdS_y = RobustScaler()
X = StdS_X.fit_transform(X)
y = StdS_y.fit_transform(y)
# Sample splitting
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25)
# Transform a vector
y_train = np.ravel(y_train)

In [ ]:
# Create the model object
regressor = SVR(kernel = 'rbf', gamma = 0.00001, C = 7742.63)
# Fit the model on the data
regressor.fit(X_train, y_train)

In [ ]:
# Get predictions 
predictions = StdS_y.inverse_transform(regressor.predict(X_test).reshape(-1,1))

In [ ]:
# Calculate MAE
print(f"MAE: {mean_absolute_error(StdS_y.inverse_transform(y_test), predictions):.4f}")

In [ ]:
# Calculate MAE
print(f"MAE standardized after the split: {mean_squared_error(StdS_y.inverse_transform(y_test), predictions):.4f}")

In [ ]:
# Calculate MAE
print(f"MAE with a robust scaler: {mean_squared_error(StdS_y.inverse_transform(y_test), predictions, squared=False):.4f}")

### Data new

In [ ]:
# Upload data
with open('Data final//data_new_imputed.pkl', 'rb') as file:
    data = pickle.load(file)

In [ ]:
# Sample 100000 observations from data
index_sample = random.sample(range(0, len(data)), 100000)
data = data.iloc[index_sample,]
# Dataframe with features and array with outcomes (shaped in 2d array)
y = data["Total revenue"].values
X = data.drop("Total revenue", axis = "columns").values
y = y.reshape(-1,1)
# Rescale and standardize 
StdS_X = RobustScaler()
StdS_y = RobustScaler()
X = StdS_X.fit_transform(X)
y = StdS_y.fit_transform(y)
# Sample splitting
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25)
# Transform a vector
y_train = np.ravel(y_train)

In [ ]:
# Create the model object
regressor = SVR(kernel = 'rbf', gamma = 0.00013, C = 599.48)
# Fit the model on the data
regressor.fit(X_train, y_train)

In [ ]:
# Get predictions 
predictions = StdS_y.inverse_transform(regressor.predict(X_test).reshape(-1,1))

In [ ]:
# Calculate MAE
print(f"MAE: {mean_absolute_error(StdS_y.inverse_transform(y_test), predictions):.4f}")

In [ ]:
# Calculate MAE
print(f"MAE standardized after the split: {mean_squared_error(StdS_y.inverse_transform(y_test), predictions):.4f}")

In [ ]:
# Calculate MAE
print(f"MAE with a robust scaler: {mean_squared_error(StdS_y.inverse_transform(y_test), predictions, squared=False):.4f}")